In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract

import os
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
imgPaths = glob('./Samples/*.jpg')

In [4]:
allBloodTestReports = pd.DataFrame(columns=['id','text'])

for imgPath in  tqdm(imgPaths,desc='BloodTestReports'):
    
    #imgPath = imgPaths[0]
    _, filename = os.path.split(imgPath)
    # extract data and text 
    image = cv2.imread(imgPath)
    data = pytesseract.image_to_data(image)
    dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
    df = pd.DataFrame(dataList[1:],columns=dataList[0])
    df.dropna(inplace=True)
    df['conf'] = df['conf'].astype(float)

    useFulData = df.query('conf >= 30')

    # Dataframe
    bloodTestReport = pd.DataFrame()
    bloodTestReport['text'] = useFulData['text']
    bloodTestReport['id'] = filename
    
    # concatenation
    allBloodTestReports = pd.concat((allBloodTestReports,bloodTestReport))

BloodTestReports: 100%|███████████████████████████████████████████████████████████████| 123/123 [02:28<00:00,  1.21s/it]


In [5]:
allBloodTestReports.to_csv('bloodTestReport.csv',index=False)